# Hyperspectral Face Authentication System

This notebook demonstrates how to use the hyperspectral face authentication system for user enrollment and verification.

## Features
- **User Enrollment**: Register new users with their hyperspectral face data
- **User Authentication**: Verify identity using 1:1 or 1:N matching
- **Database Management**: Add, remove, and list users
- **Flexible Similarity Metrics**: Support for cosine similarity and Euclidean distance
- **Threshold-based Acceptance**: Configurable thresholds for security/convenience balance

## Setup and Imports

In [ ]:
import numpy as np
import sys
import os
from pathlib import Path

# Import the authentication system
from face_authentication_system import (
    HyperspectralFaceAuthenticator,
    load_hyperspectral_image,
    create_synthetic_hyperspectral_image
)

print("✓ Imports successful")

## 1. Initialize the Authentication System

Before using the system, ensure you have:
1. Trained the model using `hyperspectral_face_recognition_model.ipynb`
2. The model file `hyperspectral_embedding_model.h5` in the current directory

In [ ]:
# Initialize the authenticator
# Using cosine similarity with threshold of 0.6
# Higher threshold = more strict authentication
authenticator = HyperspectralFaceAuthenticator(
    model_path='hyperspectral_embedding_model.h5',
    database_path='user_database.pkl',
    similarity_metric='cosine',  # 'cosine' or 'euclidean'
    threshold=0.6  # Adjust based on your security requirements
)

print("\n✓ Authentication system initialized")

## 2. Enroll New Users

Register users by providing their hyperspectral face images. Multiple images per user improve authentication accuracy.

In [ ]:
# Example: Enroll users with synthetic data
# In production, replace with real hyperspectral face images

print("Enrolling users...\n")

# User 1: Alice
alice_images = [
    create_synthetic_hyperspectral_image() for _ in range(3)
]
authenticator.enroll_user('alice', alice_images)

# User 2: Bob
bob_images = [
    create_synthetic_hyperspectral_image() for _ in range(3)
]
authenticator.enroll_user('bob', bob_images)

# User 3: Charlie
charlie_images = [
    create_synthetic_hyperspectral_image() for _ in range(3)
]
authenticator.enroll_user('charlie', charlie_images)

print(f"\nTotal enrolled users: {len(authenticator.list_users())}")

## 3. Example: Enrolling Users from Real Data

If you have the UWA HSFD dataset or similar hyperspectral face data:

In [ ]:
# Example code for enrolling from real data
# Uncomment and adapt this code when you have real data

"""
# Path to UWA HSFD dataset
DATA_PATH = './UWA_HSFD_dataset'

if os.path.exists(DATA_PATH):
    subjects = [d for d in os.listdir(DATA_PATH) 
                if os.path.isdir(os.path.join(DATA_PATH, d))]
    
    for subject_id in subjects[:5]:  # Enroll first 5 subjects
        subject_path = os.path.join(DATA_PATH, subject_id)
        image_files = [f for f in os.listdir(subject_path) 
                      if f.endswith('.npy') or f.endswith('.mat')]
        
        # Load images
        images = []
        for img_file in image_files[:3]:  # Use first 3 images
            img_path = os.path.join(subject_path, img_file)
            img = load_hyperspectral_image(img_path)
            images.append(img)
        
        # Enroll user
        authenticator.enroll_user(subject_id, images)
        
    print(f"Enrolled {len(subjects[:5])} users from dataset")
else:
    print(f"Dataset not found at {DATA_PATH}")
"""

print("Real data enrollment example (commented out)")

## 4. View Enrolled Users

In [ ]:
# List all enrolled users
users = authenticator.list_users()
print(f"Enrolled users ({len(users)}):")
for user in users:
    print(f"  - {user}")

# Get detailed info for each user
print("\nUser Details:")
for user in users:
    info = authenticator.get_user_info(user)
    print(f"  {user}: {info['num_samples']} samples, embedding_dim={info['embedding_dim']}")

## 5. Authenticate Users (1:1 Verification)

Verify a specific user's identity by comparing against their enrolled data.

In [ ]:
print("=" * 60)
print("1:1 VERIFICATION (Specific User)")
print("=" * 60)

# Test 1: Verify Alice with a similar image (should succeed)
print("\nTest 1: Verifying Alice with similar image...")
# In practice, this would be a new image of Alice
alice_test_image = alice_images[0] + np.random.randn(*alice_images[0].shape) * 0.1
authenticated, matched_user, score = authenticator.authenticate(
    alice_test_image, user_id='alice'
)
print(f"  Result: {'✓ AUTHENTICATED' if authenticated else '✗ REJECTED'}")
print(f"  User: {matched_user}")
print(f"  Similarity Score: {score:.4f}")
print(f"  Threshold: {authenticator.threshold}")

# Test 2: Try to verify Bob as Alice (should fail)
print("\nTest 2: Trying to verify Bob as Alice (imposter)...")
bob_test_image = bob_images[0]
authenticated, matched_user, score = authenticator.authenticate(
    bob_test_image, user_id='alice'
)
print(f"  Result: {'✓ AUTHENTICATED' if authenticated else '✗ REJECTED'}")
print(f"  User: {matched_user}")
print(f"  Similarity Score: {score:.4f}")
print(f"  Threshold: {authenticator.threshold}")

## 6. Authenticate Users (1:N Identification)

Identify a user from all enrolled users without specifying who they claim to be.

In [ ]:
print("=" * 60)
print("1:N IDENTIFICATION (Find Best Match)")
print("=" * 60)

# Test 1: Identify Bob from all users
print("\nTest 1: Identifying Bob from all users...")
bob_test_image = bob_images[0] + np.random.randn(*bob_images[0].shape) * 0.1
authenticated, matched_user, score = authenticator.authenticate(
    bob_test_image, user_id=None  # None = search all users
)
print(f"  Result: {'✓ IDENTIFIED' if authenticated else '✗ NOT RECOGNIZED'}")
print(f"  Matched User: {matched_user}")
print(f"  Similarity Score: {score:.4f}")
print(f"  Threshold: {authenticator.threshold}")

# Test 2: Try with unknown person (should be rejected)
print("\nTest 2: Trying with unknown person...")
unknown_image = create_synthetic_hyperspectral_image()
authenticated, matched_user, score = authenticator.authenticate(
    unknown_image, user_id=None
)
print(f"  Result: {'✓ IDENTIFIED' if authenticated else '✗ NOT RECOGNIZED'}")
print(f"  Matched User: {matched_user}")
print(f"  Similarity Score: {score:.4f}")
print(f"  Threshold: {authenticator.threshold}")

## 7. Threshold Tuning

Adjust the authentication threshold to balance security and convenience.

In [ ]:
print("Testing different thresholds...\n")

test_image = charlie_images[0] + np.random.randn(*charlie_images[0].shape) * 0.1

thresholds = [0.3, 0.5, 0.7, 0.9]

for thresh in thresholds:
    authenticator.update_threshold(thresh)
    authenticated, matched_user, score = authenticator.authenticate(
        test_image, user_id='charlie'
    )
    status = '✓ PASS' if authenticated else '✗ FAIL'
    print(f"Threshold {thresh:.1f}: {status} (score: {score:.4f})")

# Reset to original threshold
authenticator.update_threshold(0.6)

## 8. Database Management

In [ ]:
print("Database Management Operations\n")

# Add a new user
print("1. Adding new user 'david'...")
david_images = [create_synthetic_hyperspectral_image() for _ in range(2)]
authenticator.enroll_user('david', david_images)

# List users
print(f"\n2. Current users: {authenticator.list_users()}")

# Remove a user
print("\n3. Removing user 'david'...")
authenticator.remove_user('david')

# List users again
print(f"\n4. Users after removal: {authenticator.list_users()}")

# Export database
print("\n5. Exporting database...")
authenticator.export_database('backup_database.pkl')

print("\n✓ Database management operations completed")

## 9. Performance Analysis

Evaluate authentication performance with multiple test cases.

In [ ]:
import time

print("Performance Analysis\n")
print("=" * 60)

# Test authentication speed
test_image = alice_images[0]
num_iterations = 10

start_time = time.time()
for _ in range(num_iterations):
    authenticator.authenticate(test_image, user_id='alice')
end_time = time.time()

avg_time = (end_time - start_time) / num_iterations
print(f"Average authentication time: {avg_time*1000:.2f} ms")
print(f"Throughput: {1/avg_time:.2f} authentications/second")

# Test accuracy with synthetic variations
print("\n" + "=" * 60)
print("Testing robustness to variations...\n")

noise_levels = [0.05, 0.1, 0.2, 0.3]
user_to_test = 'alice'
base_image = alice_images[0]

for noise in noise_levels:
    successes = 0
    num_tests = 20
    
    for _ in range(num_tests):
        noisy_image = base_image + np.random.randn(*base_image.shape) * noise
        authenticated, _, _ = authenticator.authenticate(noisy_image, user_id=user_to_test)
        if authenticated:
            successes += 1
    
    accuracy = successes / num_tests * 100
    print(f"Noise level {noise:.2f}: {accuracy:.1f}% authentication rate")

print("\n✓ Performance analysis completed")

## 10. Integration Example

Example of how to integrate the authentication system into a larger application.

In [ ]:
def authentication_pipeline(image_path: str, user_id: str = None):
    """
    Complete authentication pipeline from image path.
    
    Args:
        image_path: Path to hyperspectral face image
        user_id: Optional user ID for 1:1 verification
    
    Returns:
        Dictionary with authentication results
    """
    try:
        # Load image
        # In practice: image = load_hyperspectral_image(image_path)
        image = create_synthetic_hyperspectral_image()  # Placeholder
        
        # Authenticate
        authenticated, matched_user, score = authenticator.authenticate(
            image, user_id=user_id
        )
        
        # Prepare result
        result = {
            'success': True,
            'authenticated': authenticated,
            'user_id': matched_user,
            'confidence': score,
            'threshold': authenticator.threshold,
            'message': f"User {matched_user} authenticated" if authenticated else "Authentication failed"
        }
        
        return result
        
    except Exception as e:
        return {
            'success': False,
            'authenticated': False,
            'error': str(e),
            'message': 'Authentication error'
        }

# Test the pipeline
print("Testing authentication pipeline...\n")

result = authentication_pipeline('path/to/image.npy', user_id='alice')
print("Pipeline Result:")
for key, value in result.items():
    print(f"  {key}: {value}")

print("\n✓ Integration example completed")

## Summary

This notebook demonstrated:

1. **Initialization**: Setting up the authentication system with a pre-trained model
2. **User Enrollment**: Registering users with multiple face samples
3. **1:1 Verification**: Authenticating a specific user claim
4. **1:N Identification**: Identifying users from a database
5. **Threshold Tuning**: Adjusting security/convenience balance
6. **Database Management**: Adding, removing, and exporting user data
7. **Performance Analysis**: Evaluating speed and robustness
8. **Integration**: Building complete authentication pipelines

### Next Steps

- Replace synthetic data with real hyperspectral face images
- Fine-tune the threshold based on your security requirements
- Integrate with your application's user management system
- Add logging and monitoring for production deployment
- Implement additional security measures (liveness detection, anti-spoofing)